# Download and manipulate CMIP6 data

In [21]:
import glob
import os
import requests
import zipfile

import netcdf_scm.io
import pandas as pd
from scmdata import ScmRun, run_append
import tqdm

In [4]:
DATA_DIR = os.path.join("..", "data_input", "cmip6")
DATA_DIR

'../data_input/cmip6'

In [7]:
zip_file = os.path.join(".", "cmip6_data.zip")

In [10]:
experiments_to_download = (
#    "historical",
    "ssp245",
)

variables_to_download = (
    "tas",
)

if not os.path.isdir(DATA_DIR):
    print("Downloading relevant data")
    for exp in tqdm.tqdm(experiments_to_download, desc="experiment"):
        for var in tqdm.tqdm(variables_to_download, desc="variable"):
            params = (
                ("experiment_id", exp),
                ("variable_id", var),
                ("normalised", "21-yr-running-mean"),
                ("timeseriestype", "average-year-mid-year"),
            )
            url = "https://cmip6.science.unimelb.edu.au/api/v1/download_zip"
            r = requests.get(url, params=params)
            r.raise_for_status()

            with open(zip_file, "wb") as f:
                f.write(r.content)

            zip_ref = zipfile.ZipFile(zip_file, "r")
            zip_ref.extractall(DATA_DIR)
            zip_ref.close()

experiment: 100%|████████████████████████████████| 1/1 [02:02<00:00, 122.55s/it]


In [14]:
ssp245_tas_files = sorted(
    glob.glob(
        os.path.join(
            DATA_DIR,
            "**",
            "average-year-mid-year",
            "**",
            "NORMED*tas*.MAG",
        ),
        recursive=True,
    )
)
len(ssp245_tas_files)

193

In [34]:
out_table = []
load_kwargs = dict(drs="CMIP6Output")

for f in tqdm.tqdm(ssp245_tas_files):

#     db = []
    tas = netcdf_scm.io.load_mag_file(f, **load_kwargs).filter(region="World")
    assert tas.metadata["normalisation method"] == "21-yr-running-mean"

    climate_model = tas.get_unique_meta("climate_model", no_duplicates=True)
    member_id = tas.get_unique_meta("member_id", no_duplicates=True)
#     print(tas)
    
    out_table.append(tas)

# db = run_append(db)

#     out_dict = {
#         "climate_model": climate_model,
#         "member_id": member_id,
#         "mip_era": db.get_unique_meta("mip_era", no_duplicates=True),
#     }

#     for k, v in out_dict.items():
#         try:
#             val = v.magnitude
#             unit = v.units
#             key = "{} ({})".format(k, unit)
#         except AttributeError:
#             key = k
#             val = v

#         if key in out_table:
#             out_table[key].append(val)
#         else:
#             out_table[key] = [val]

# out_table = pd.DataFrame(out_table)

100%|█████████████████████████████████████████| 193/193 [00:10<00:00, 17.95it/s]


In [38]:
out_table = run_append(out_table)

In [39]:
out_table.head()

time                                                                                   1850-07-01  \
activity_id climate_model member_id mip_era model       region scenario unit variable               
ScenarioMIP AWI-CM-1-1-MR r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.123190   
            FGOALS-g3     r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.153702   
                          r2i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.153702   
                          r3i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.153702   
                          r4i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.153702   

time                                                                                   1851-07-01  \
activity_id climate_model member_id mip_era model       region scenario unit variable               
ScenarioMIP AWI-CM-1-1-MR r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.162130   
            FGOALS-g3     r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.050636   
                          r2i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.050636   
                          r3i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.050636   
                          r4i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.050636   

time                                                                                   1852-07-01  \
activity_id climate_model member_id mip_era model       region scenario unit variable               
ScenarioMIP AWI-CM-1-1-MR r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.149941   
            FGOALS-g3     r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.013877   
                          r2i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.013877   
                          r3i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.013877   
                          r4i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.013877   

time                                                                                   1853-07-01  \
activity_id climate_model member_id mip_era model       region scenario unit variable               
ScenarioMIP AWI-CM-1-1-MR r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.088612   
            FGOALS-g3     r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.026406   
                          r2i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.026406   
                          r3i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.026406   
                          r4i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.026406   

time                                                                                   1854-07-01  \
activity_id climate_model member_id mip_era model       region scenario unit variable               
ScenarioMIP AWI-CM-1-1-MR r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.103306   
            FGOALS-g3     r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.025644   
                          r2i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.025644   
                          r3i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.025644   
                          r4i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.025644   

time                                                                                   1855-07-01  \
activity_id climate_model member_id mip_era model       region scenario unit variable               
ScenarioMIP AWI-CM-1-1-MR r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas         0.120271   
            FGOALS-g3     r1i1p1f1  CMIP6   unspecified World  ssp245   K    tas        -0.070094   
                          r2i1p1f1  CMIP6   unspecified 

In [49]:
# de-junk
out_table = out_table.timeseries()

In [52]:
out_table_means = out_table.groupby('climate_model').mean()
out_table_means

time,1850-07-01,1851-07-01,1852-07-01,1853-07-01,1854-07-01,1855-07-01,1856-07-01,1857-07-01,1858-07-01,1859-07-01,...,2091-07-01,2092-07-01,2093-07-01,2094-07-01,2095-07-01,2096-07-01,2097-07-01,2098-07-01,2099-07-01,2100-07-01
climate_model,,,,,,,,,,,,,,,,,,,,,
ACCESS-CM2,0.012021,0.014690,-0.025660,0.014244,0.011469,-0.032908,-0.045573,0.036360,0.011060,0.016916,...,3.264263,3.303120,3.258563,3.294217,3.275740,3.396337,3.347633,3.356540,3.356443,3.378940
ACCESS-ESM1-5,0.082331,0.063298,-0.030325,0.078419,0.153294,0.034055,0.095915,0.031679,0.015508,0.038038,...,3.108643,3.146743,3.166507,3.201133,3.167670,3.205203,3.106107,3.057073,3.118143,3.182930
AWI-CM-1-1-MR,-0.123190,-0.162130,-0.149941,-0.088612,0.103306,0.120271,-0.049101,-0.178200,-0.057063,-0.034576,...,2.790770,2.885880,3.199580,3.029180,2.949430,2.934760,2.958430,2.788310,2.803140,2.728520
CESM2,-0.071711,-0.018635,0.066435,-0.033800,-0.027930,0.003034,-0.045934,-0.176260,-0.057909,0.040286,...,3.296847,3.359610,3.285470,3.325703,3.327410,3.320140,3.405880,3.292460,3.332263,3.501050
CESM2-WACCM,-0.007388,-0.167219,-0.044075,-0.072160,-0.126604,-0.040556,0.033457,0.001567,-0.133822,0.013627,...,3.127264,3.241120,3.280610,3.166578,3.259208,3.275980,3.218180,3.355670,3.305712,3.371228
CMCC-CM2-SR5,0.155986,0.212930,0.081849,0.365750,0.321307,0.323679,0.416980,0.446327,0.399858,0.321027,...,4.264430,4.067930,4.030200,4.115580,4.143140,4.297380,4.242530,4.208610,4.330330,4.202420
CMCC-ESM2,0.060162,0.171146,0.077076,0.064636,0.092505,0.244028,0.310062,0.206949,0.209377,0.303318,...,3.630310,3.795930,4.067310,4.021300,3.884650,3.841390,3.928080,3.971190,3.999520,3.801130
CNRM-CM6-1,-0.037678,-0.080929,-0.072735,-0.038629,-0.043558,-0.026948,-0.066927,0.018007,0.031398,0.014664,...,3.229203,3.269717,3.246305,3.286088,3.313800,3.385165,3.320107,3.354680,3.331033,3.405877
CNRM-CM6-1-HR,-0.152081,-0.206964,-0.149775,-0.096587,-0.094674,0.001441,-0.098072,-0.100262,-0.158562,-0.057563,...,3.702820,3.547820,3.541150,3.676020,3.685180,3.580990,3.656790,3.773900,3.772440,3.645400


In [53]:
out_table_means.to_csv('../data_output/ssp245.csv')